# XGBoost for BACtrack Data

In [1]:
import os
import logging
import datetime as dt
import pandas as pd
import numpy as np
import h5py
import itertools
import sys

import pyarrow as pa
import pyarrow.parquet as pq

# Might need to go before xgboost imports
import importlib
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import xgboost as xgb
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score, auc, roc_curve, \
    confusion_matrix, classification_report
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
%matplotlib inline
%config Application.log_level="INFO"

In [2]:
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

logging.info('Hello world!')

2019-06-09 20:01:42,795 | INFO : Hello world!


#### GLOBAL VARIABLES AND SETTINGS

In [3]:
# Secure-copy updated data to server: 
# scp $LOCAL_PATH kaschbacher@tisoncluster.ucsf.edu:$SERVER_PATH

In [4]:
DATA_FOLDER = 'data'
FIGURE_FOLDER = 'figures'
#FILENAME = 'bac_2018-11-27.h5'
FILENAME = 'bac_2019-06-07.parquet'#'bac_2019-03-26.parquet'
PATH = '/'.join([DATA_FOLDER, FILENAME])
TODAY = str(dt.date.today())
MISSING_VALUE = np.nan

In [5]:
SEED = 9
TRAIN_P = .70
DEV_P = .10

TEST_P = 1 - TRAIN_P - DEV_P
split_percents = [TRAIN_P, DEV_P, TEST_P]

In [6]:
DEFAULT_BOOSTING_PARAMS = {
    'silent':1,
    'learning_rate':.3,
    'max_depth':6,
    'min_child_weight':1,# default=1, larger=more conservative; building process will give up partitioning 
    'subsample':0.80,
    'colsample_bytree':0.80,
    'objective':'binary:logistic',#'objective':'binary:logistic' or 'reg:linear'
    'num_boost_round': 10,
    #'eval_metric':'auc',
    'random_state':7,
    'lambda':1,# default=1, L2 (ridge); higher values = more conservative
    'alpha':0,# default=0, L1 (lasso); higher values = more conservative
    'gamma':0,# [default=0, alias: min_split_loss] - Larger=more conservative. Minimum loss reduction required to make a further partition on a leaf node of the tree
    'scale_pos_weight':1# default=1, reset later based on class distribution
}

In [7]:
# Starts with defaults, but can change
boosting_params = DEFAULT_BOOSTING_PARAMS

In [8]:
# Set Parameters for Grid Search
def get_gs_params(params):
    gs_params = {
        'learning_rate': params['learning_rate'], 
        'num_boost_round': params['num_boost_round'],
        #'n_estimators': best_n_estimators, 
        'max_depth': params['max_depth'],
        'min_child_weight': params['min_child_weight'], 
        'gamma': params['gamma'], 
        'subsample': params['subsample'], 
        'colsample_bytree': params['colsample_bytree'], 
        'return_train_score': True,
        'objective': 'binary:logistic', 
        #'n_jobs': 4,# -1 --> make use of all the cores in your system; but supposedly could also slow things 
        'scale_pos_weight':params['scale_pos_weight'], 
        'seed':7
    }
    return gs_params

In [9]:
figure_params = {
    'figure.figsize':(5,5),
    'figure.titlesize':20,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'axes.labelsize': 18
}

#### Load Data

In [10]:
def load_data(path):
    df = pq.read_table(path, use_threads=4).to_pandas()
    names = df.columns.tolist()
    data = df.values
    return names, data

#### Randomly Shuffle the rows
--- so that there is no systematic association of date with train/val/test

In [11]:
def shuffle_rows(data, SEED):
    # shuffles rows of an np-array, w/ seed, order will always be replicable
    np.random.seed(seed=SEED)
    np.random.shuffle(data)
    return data

#### Option to partition by observation --> train/dev/test

In [50]:
def partition_by_observation(split_percents, data, idx_uid):
    train_per, dev_per, test_per = split_percents
    m = data.shape[0]
    train, dev, test = np.split(data, [int(m*train_per), int(m*(train_per+dev_per))]) 
    print ('train shape: {}, dev shape: {}, test shape: {}'.format(train.shape, dev.shape, test.shape))
    assert train.shape[0] + dev.shape[0] + test.shape[0] == data.shape[0]
    
    # Assumes y or target outcome is the first column, and that all other columns will be included in X
    # Omit user_id in column 1
    if idx_uid!=1: print ('Error: Expected user id in column 1, in function partition by observation')
    partitions = {}
    partitions['X_train'] = train[:,2:]
    partitions['y_train'] = train[:,0]
    partitions['X_dev'] = dev[:,2:]
    partitions['y_dev'] = dev[:,0]
    partitions['X_test'] = test[:,2:]
    partitions['y_test'] = test[:,0]
    
    return partitions

#### Option to Partition by User
<p> -- dev and test set will not share any users with train set. No learning possible about individual users </p>

In [13]:
def get_unique_uids(data):
    # Get unique user ids
    user_ids = np.unique(data[:,1])
    print ('Number of unique users: {}'.format(user_ids.shape[0]))
    np.random.shuffle(user_ids)
    return user_ids

In [14]:
def partition_users(users, split_percents):
    train_per, dev_per, test_per = split_percents
    s = len(users)
    arys = np.split(users, [int(train_per*s), int((train_per+dev_per)*s)])
    return arys[0], arys[1], arys[2]

In [15]:
def get_idx(uuids, idx_uid):
    # given a list of user ids, find the indices in the data np-ary that represent observations from these users
    idx = []
    for ui in uuids:
        temp = np.where(data[:, idx_uid]==ui)[0]
        idx = np.append(idx, temp, axis=0).astype(int)
    return idx

In [54]:
def partition_by_user(split_percents, data, idx_uid):
    # Get arrays of user_ids that partition up the train, dev, test
    uuids = get_unique_uids(data)# returns a randomly shuffled array of unique user ids
    train_ui, dev_ui, test_ui = partition_users(uuids, split_percents)
    assert len(uuids) == train_ui.shape[0] + dev_ui.shape[0] + test_ui.shape[0]

    # Get the indices for the observations belonging to a given set of user_ids

    partitions={}
    segments = {'train': train_ui, 'dev': dev_ui, 'test': test_ui}
    for k, uuids in segments.items():
        print ('Running segment: {}...'.format(k))
        %time
        idx = get_idx(uuids, idx_uid)
        idx = np.sort(idx)
        partitions['X_'+k]=data[idx, 2:]
        partitions['y_'+k]=data[idx, 0]
    print ('train shape: {}, dev shape: {}, test shape: {}'.format(partitions['X_train'].shape, partitions['X_dev'].shape, partitions['X_test'].shape))
        
    return partitions

#### Package Data Matrices for XGBoost

In [47]:
def package_data (partitions, names, idx_uid):
    # Assign data - (Note: must dropna first, otherwise missing=-999 throws an error because NaNs also still exist)
    print ("\nSetting up data-matrices for Gradient Boosted Classification Tree with Outcome: {}...\n".format(names[0]))

    names = [str(name) for name in names]# convert from byte to string
    del names[idx_uid]# Omit user_id at idx, 1
    partitions['names']=names
    partitions['dtrain'] = xgb.DMatrix(data=partitions['X_train'], label=partitions['y_train'], 
                                       feature_names=names[1:], missing=MISSING_VALUE)
    partitions['ddev'] = xgb.DMatrix(data=partitions['X_dev'], label=partitions['y_dev'], 
                                     feature_names=names[1:], missing=MISSING_VALUE)
    partitions['dtest'] = xgb.DMatrix(data=partitions['X_test'], label=partitions['y_test'], 
                                      feature_names=names[1:], missing=MISSING_VALUE)
    
    return partitions

#### Balance Classes

In [31]:
def get_balance_weight(y_train):

    pos = np.sum(y_train, axis=0, dtype=int)
    neg = y_train.shape[0]-pos
    print ('\n{} positive cases and {} negative cases'.format(pos, neg))

    scale_pos_weight = round(neg/pos, 2)# 2.38 for the BACtrack dataset
    print ('Scale Weight for balanced classes would be: {}'.format(scale_pos_weight))
    return scale_pos_weight

#### Training/Fitting the Model
--- fit vs train -- https://datascience.stackexchange.com/questions/17282/xgbregressor-vs-xgboost-train-huge-speed-difference

In [32]:
def fit_model(dtrain, ddev, boosting_params):
    # # Training a Model
    # # -- xgboost.train will ignore parameter n_estimators, while xgboost.XGBRegressor accepts. 
    # # -- In xgboost.train, boosting iterations (i.e. n_estimators) is controlled by num_boost_round(default: 10)
    # # -- the learning_rate (eta) and num_boost_rounds are instrisincally connected.  
    # # -- Slower learning rates will typically require more rounds

    # if num_boost_round not explicitly given as kwarg, it will be overridden by default of 10, which can be seen in eval printing
    watchlist = [(ddev, 'dev'), (dtrain, 'train')]
    %time
    model = xgb.train(params=boosting_params, dtrain=dtrain, evals=watchlist, verbose_eval=True,
                      num_boost_round=boosting_params['num_boost_round'])# try predictor='gpu_predictor'
    return model

#### Make & Evaluate Model Predictions

In [33]:
def predict(model, d_true):
    # d_true is the xgboost prepared dataset corresponding to test or dev (per needs); includes X and y and names
    y_proba = model.predict(d_true)
    y_pred = [round(value) for value in y_proba]# Use to get 0 vs 1?
    return y_pred, y_proba

In [34]:
def evaluate_model(y_true, y_pred, y_proba):
    accuracy = accuracy_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_proba, average='micro', max_fpr=None)#sample_weight=scale_pos_weight, 
    print ('Accuracy: {:02.2f}%'.format(accuracy*100.))
    print ('ROC AUC: {:02.2f}%'.format(roc_auc*100.))
    return accuracy, roc_auc

#### Confusion Matrix

In [35]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix\n")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')

#### Full Report (w/ Confusion Matrix)

In [36]:
def full_report(figure_params, y_test, y_pred, target_names):
    
    # # Plot Confusion Matrix for Classification - http://scikit-learn.org/stable/modules/model_evaluation.html
    plt.rcParams.update(figure_params)

    cnf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    fig = plt.figure()
    plot_confusion_matrix(cnf_matrix, ['low BAC','high BAC'], normalize=True)
    plt.title('Confusion Matrix', fontsize=16)
    plt.show()
    fig.savefig('/'.join( \
        [FIGURE_FOLDER,'/cm/Normalized_Confusion_Matrix_BAC_Classification_{}.png'.format(TODAY)]), \
        bbox_inches='tight')

    # Classification Report
    #print ('\nAccuracy: {:02.2f}%\n'.format(accuracy*100.))
    print (classification_report(y_test, y_pred, target_names=target_names),'\n')

### Plot ROC

In [37]:
def plot_ROC(y_test, y_pred_proba):
    # y_test can also be y_dev, depending on context
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # Calculate x and y for ROC-curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    #print (fpr.shape, tpr.shape, roc_auc)# DEBUG

    #Plot of a ROC curve for a specific class
    plt.rcParams.update(figure_params)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

### Built-in Feature Importance Graph:  
--- importance_type='weight' matches above.  'cover' provides quite different results, which may be of interest to report in paper.

In [38]:
def graph_importances(model):  
    
    # -- http://xgboost.readthedocs.io/en/latest/python/python_api.html
    # weight” is the number of times a feature appears in a tree “gain” is the average gain of splits which use the feature...
    # “cover” is the average coverage of splits which use the feature, where coverage is defined as the number of samples affected by the split
    plt.rcParams['figure.figsize']=(14,18)
    xgb.plot_importance(model, importance_type='weight')
    plt.title('Feature Importance (Weight)', fontsize=20)
    plt.show()
    plt.savefig('/'.join([FIGURE_FOLDER,'fi/feature_importance_{}.png'.format(TODAY)]), \
                bbox_inches='tight')   

### MAIN

In [39]:
%%time
# Load Data
names, data = load_data(PATH)
print (data.shape, '\n')# QA
print (names[:3])# Ensure that bac_clinical (or the outcome) is the first column!

(973264, 92) 

['bac_clinical', 'user_id', 'bac_guess']
CPU times: user 4.22 s, sys: 3.78 s, total: 8 s
Wall time: 5.14 s


#### Prepare Data

In [40]:
%%time
# Partition randomly
shuffle_rows(data, SEED)

CPU times: user 4.03 s, sys: 23 ms, total: 4.05 s
Wall time: 4.02 s


array([[ 1.00000000e+00,  2.79300000e+03,  0.00000000e+00, ...,
                    nan,             nan,             nan],
       [ 1.00000000e+00,  2.99050000e+04,  5.00000007e-02, ...,
         2.19900000e+03,  5.60000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  1.73040000e+04,  3.99999991e-02, ...,
        -4.80000000e+01, -1.00000000e+00,  1.00000000e+00],
       ...,
       [ 1.00000000e+00,  4.34900000e+03,  2.99999993e-02, ...,
                    nan,             nan,             nan],
       [ 1.00000000e+00,  2.10500000e+03,  2.99999993e-02, ...,
                    nan,             nan,             nan],
       [ 1.00000000e+00,  1.54300000e+03,  0.00000000e+00, ...,
         1.17000000e+02,  2.10000000e+00,  1.00000000e+00]])

#### Partition data in 1 of 2 ways - (by randomly assigning either observations or users to the train/dev/test)

In [51]:
# Confirm what column user_id is in
idx_uid = 1
assert names[idx_uid]=='user_id'
    
partitions = partition_by_user(split_percents, data, idx_uid)
#partitions = partition_by_observation(split_percents, data, idx_uid)

# Package data for XGboost
partitions = package_data(partitions, names, idx_uid)# Adds names + dtrain, ddev, dtest

Number of unique users: 33460
Running segment: train...
CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 39.8 µs
Running segment: dev...
CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs
Running segment: test...
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.3 µs

Setting up data-matrices for Gradient Boosted Classification Tree with Outcome: bac_clinical...



### Balancing the Classes/ Determining Class Weights

In [ ]:
def balance_classes(boosting_params, partitions):
    prior_scale_pos_weight = boosting_params['scale_pos_weight']
    # Reset scale weight in boosting params and in the grid search params both
    boosting_params['scale_pos_weight'] = get_balance_weight(partitions['y_train'])
    gs_params = get_gs_params(boosting_params)
    print ('\nResetting positive scale weight from {} to {}...\n'.format(prior_scale_pos_weight, boosting_params['scale_pos_weight']))
    return gs_params

## Default Model for Comparison

In [ ]:
def base_model_wwo_balancing(boosting_params, partitions):
    
    # Assumption is that boosting_params starts out at the default, and then can be altered throughout the tuning process
    if boosting_params != DEFAULT_BOOSTING_PARAMS:
        print ('Resetting boosting params to be equal to DEFAULT\n')
        boosting_params = DEFAULT_BOOSTING_PARAMS
    # Assumes we start with no scale weighting, and compare that to weighting inversely to case prevalence in the training set
    if boosting_params['scale_pos_weight'] != 1:
        print ('Converting the scale weight for boosting params back to 1 for the first run...\n')
        boosting_params['scale_pos_weight'] = 1
    
    # Fit & Evaluate model twice: 1) Unbalanced, 2) Balanced
    for i in range(0,2):
        if i>0: 
            print ('-'*60,'\n')
            balance_classes(boosting_params, partitions)
            
        # Fit Model
        print ('Run {}, Scale weight = {:0.2f}'.format(i, boosting_params['scale_pos_weight']))
        model= fit_model(partitions['dtrain'], partitions['ddev'], boosting_params)

        # Predict/ Evaluate in Dev set
        y_true = partitions['y_dev']; d_true = partitions['ddev']
        y_pred, y_proba = predict(model, d_true)
        accuracy, roc = evaluate_model(y_true, y_pred, y_proba)

        # Get Results & Figures
        plot_ROC(y_true, y_proba)
        full_report(figure_params, y_true, y_pred, ['bac <.08', 'bac >=.08'])
        #graph_importances(model) 
        
#base_model_wwo_balancing(boosting_params, partitions)   

### Use Cross-Validated XGBoost to determine the optimal number of trees

In [ ]:
%%time

# EVALUATE LEARNING RATE AND MAX_DEPTH TOGETHER
# For learning_rate=1 & max_depth=6 --> optimal number of trees=30
boosting_params['learning_rate']= .01
boosting_params['max_depth']= 6
boosting_params['nthread']=-1
boosting_params['num_boost_round']= 500
boosting_params['verbose']=2
# watchlist = [(partitions['ddev'], 'dev'), (partitions['dtrain'], 'train')]
# boosting_params['evals']=watchlist

print ('Boosting Params...\n')
for k, p in boosting_params.items():
    print (k,p)
print ('\n')

cvresult = xgb.cv(boosting_params, partitions['dtrain'], num_boost_round=boosting_params['num_boost_round'], \
                 nfold=3, early_stopping_rounds=10, verbose_eval=True, \
                 seed=7, metrics='auc')# metrics=['auc','mae','map']

best_n_estimators = cvresult.shape[0]
print ('\nThe optimal number of trees is {}\n'.format(best_n_estimators))
# optimal trees was 52 with learning rate of 1 and early_stopping=10

In [ ]:
best_idx = np.where(cvresult.iloc[:,2]==cvresult.iloc[:,2].max())[0][0]
print ('Best AUC in test set: {:0.4f} with {} trees or n_estimators'.format(cvresult.iloc[best_idx,2], best_idx+1))

### Reset the number of Estimators (Trees) to match CV outcome

In [ ]:
boosting_params['num_boost_round']=best_n_estimators
gs_params['num_boost_round']=best_n_estimators

print (boosting_params)
today = dt.date.today()
print (today)

# Grid Search

In [ ]:
# %%time

param_test1 = {
 'max_depth': list(np.arange(5, 8, 1)),
 'min_child_weight': list(np.arange(1, 21, 10))#[ 1  6 11 16 21]
}# 'alpha': [0, 1, 3]

boosting_params['learning_rate']= .3
boosting_params['num_boost_round']= 80
boosting_params['pos_scale_weight']= get_balance_weight(partitions['y_train'])

# Prints out in tmux terminal (not sure why...)
gs_params = get_gs_params(boosting_params)
print ('\n\nGrid Search parameters...\n','-'*20,'\n')
for k, gsp in gs_params.items():
    print (k, gsp)
print ('\n\n')
print (param_test1)

In [ ]:
%%time

# # Send logging to a file
today = dt.date.today()
#sys.stdout = open('xgboost_output_{}.txt'.format(today), 'w')

# May get an Unpickling Error/ Broken Process if using custom score with multiprocessing/n_jobs # scoring='roc_auc', 
gsearch1 = GridSearchCV(estimator = XGBClassifier(gs_params), \
                        param_grid = param_test1, \
                       verbose=3, \
                        n_jobs=1, iid=False, cv=3, refit=True)#return_train_score=True,

In [ ]:
%env LOKY_PICKLER='cloudpickle' 
import multiprocessing
#multiprocessing.set_start_method('forkserver', force=True)

try:
    multiprocessing.set_start_method('spawn')
except RuntimeError:
    pass

In [ ]:
gsearch1.fit(partitions['X_train'], partitions['y_train'], eval_metric='auc', verbose=True, \
            eval_set=[(partitions['X_dev'], partitions['y_dev'])], early_stopping_rounds=10)
# # Logging output to file
#sys.stdout = sys.__stdout__

In [ ]:
# PRINT BEST VALUES
# Note: The best score from GridSearchCV is biased. :( If you gridsearch best params, you will overfit in terms of AUC
print ('Best Hyperparameters: ', gsearch1.best_params_)
print ('Best Score: {:02f}'.format(gsearch1.best_score_))
print ('Best Estimator: ',gsearch1.best_estimator_)

### Reset Parameters to Best Values (from GridSearch)

In [ ]:
new_params = list(gsearch1.best_params_.keys())
for k, v in gsearch1.best_params_.items():
    print (k,v)
    boosting_params[k]=v  

In [ ]:
print (boosting_params)
print (DEFAULT_BOOSTING_PARAMS)

In [ ]:
#pd.DataFrame(gsearch1.cv_results_).head(3)

### Add Regularization

In [ ]:
# boosting_params['alpha']=1
# boosting_params['lambda']=10
# print (boosting_params)

# RUN FINAL MODEL

In [ ]:
# Fit Model - evaluates in the test set
boosting_params['learning_rate']= .03
boosting_params['max_depth']=8
boosting_params['nthread']=-1
boosting_params['num_boost_round']= 500
boosting_params['verbose']=2
balance_classes(boosting_params, partitions)


print ('Boosting Params...\n')
for k, p in boosting_params.items():
    print (k,p)
print ('\n')

# error readout corresponds to 1-accuracy
model= fit_model(partitions['dtrain'], partitions['ddev'], boosting_params)
print ('\n', boosting_params)

In [ ]:
# Make Predictions in the TEST set
y_pred, y_proba = predict(model, partitions['dtest'])
accuracy, roc_auc = evaluate_model(partitions['y_test'], y_pred, y_proba)

# Plot Confusion Matrix
target_names = ['bac <.08', 'bac >=.08']
full_report(figure_params, partitions['y_test'], y_pred, target_names)

### ROC-AUC
--- https://www.kaggle.com/jeremy123w/xgboost-with-roc-curve
--- # # http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py

In [ ]:
plot_ROC(partitions['y_test'], y_proba)

## Feature Importances

In [ ]:
graph_importances(model) 

# Save MODEL object - pickle

In [ ]:
import pickle
pickle.dump(model, open("bac_final.pickle.dat", "wb"))

In [ ]:
print (DEFAULT_BOOSTING_PARAMS)
print (boosting_params)

In [ ]:
# ### Try to save to Kirstin's local

# LOCAL_FIGURE_FOLDER = '/Users/KAschbacher/Desktop/eheart/analysis/BAC/figures/server'
# plt.savefig('/'.join([LOCAL_FIGURE_FOLDER,'feature_importance_{}.png'.format(TODAY)]), \
#             bbox_inches='tight')